In [9]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from utils.helpers import *
from utils.preprocessing import *
from implementations.least_squares import least_squares
import datetime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
y_train, tx_train, ids_train = load_csv_data('data/train.csv')
y_test, tx_test, ids_test = load_csv_data('data/test.csv')

[ 1. -1. -1. ...  1. -1. -1.]


In [24]:
max_iters = 100
gamma = 0.000001

# Initialization
w_initial = np.array([0] * 30)

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y_train, tx_train, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/99): loss=0.5000000000000002, w0=0.00010766649510400011, w1=-2.7284198603999987e-05
Gradient Descent(1/99): loss=1.8007092524629227, w0=2.3971414374319977e-05, w1=3.523635396779801e-05
Gradient Descent(2/99): loss=31.101752442252863, w0=0.0007710648697456105, w1=-0.00032521856702279255
Gradient Descent(3/99): loss=688.0300820896401, w0=-0.002474078721952218, w1=0.001318897475978849
Gradient Descent(4/99): loss=15414.27594333, w0=0.013135299159148729, w1=-0.006526189710631012
Gradient Descent(5/99): loss=345528.428187897, w0=-0.06056322554527148, w1=0.030558075576377533
Gradient Descent(6/99): loss=7745604.319635325, w0=0.2885459492568912, w1=-0.14508019467290945
Gradient Descent(7/99): loss=173631018.6211872, w0=-1.3642096415793699, w1=0.6864451133452817
Gradient Descent(8/99): loss=3892237567.6028767, w0=6.461100443635003, w1=-3.250575106470919
Gradient Descent(9/99): loss=87251191844.92096, w0=-30.588739368440102, w1=15.389702645907896
Gradient Descent(10/99): loss

KeyboardInterrupt: 

In [48]:
x_train_sep, y_train_sep, ids_train_sep, indx_train_sep = split_data_by_DER_mass_MMC(tx_train, y_train, ids_train)
x_test_sep, y_test_sep, ids_test_sep, indx_test_sep = split_data_by_DER_mass_MMC(tx_test, y_test, ids_test)

In [49]:
w = {}
for i in range(8):
    w[i] = least_squares(tx=x_train_sep[i], y=y_train_sep[i])
w

{0: array([ 7.05315584e-03, -1.00107038e-02, -1.71017660e-02, -5.86254978e+01,
         2.85433819e-01,  5.86239256e+01, -6.82673011e+00, -6.72683550e-01,
         5.08793036e-04,  6.82716233e+00,  6.49148147e-04, -2.40854338e-06,
         6.84583685e+00,  3.25882195e-03,  3.23392847e-04,  1.00892516e-03,
        -3.91653046e-03,  2.80340854e-04]),
 1: array([-6.12944951e-03, -1.36786304e-03,  8.66049259e+01,  1.28246531e-02,
        -8.66118009e+01,  9.69218893e+00, -4.08122931e-01,  6.00473396e-01,
        -9.69778349e+00, -4.10739787e-04, -2.18040318e-03, -9.67107778e+00,
         4.67166021e-03,  3.30285783e-04,  1.11635273e-02,  1.41706985e-03,
        -8.93675060e-04]),
 2: array([ 3.19847240e-03, -7.94525750e-03, -7.09955614e-03, -1.39409788e-03,
         7.28466247e-02,  1.94402395e-03, -6.60896970e+00, -3.18039240e-01,
         9.01289299e-02,  6.61169745e+00,  1.35130651e-03, -3.68202975e-03,
         6.62095725e+00, -5.99707962e-03, -5.65678721e-04,  5.98797795e-04,
        

In [50]:
len(tx_train)

250000

In [51]:
for i in range(8):
    predictions = predict_labels(w[i], x_test_sep[i])
    y_test[indx_test_sep[i]] = predictions

In [45]:
len(indx_test_sep[0])

568238

In [46]:
import pandas as pd

df = pd.read_csv("output2.csv")
df['New Prediction'] = y_test

In [47]:
df[df['New Prediction']!=df['Prediction']]

,Id,Prediction,New Prediction


In [54]:
ids_test_sep

{0: array([350002, 350003, 350005, ..., 918232, 918235, 918237]),
 1: array([350000, 350014, 350015, ..., 918193, 918218, 918220]),
 2: array([350001, 350006, 350010, ..., 918227, 918231, 918236]),
 3: array([350034, 350037, 350049, ..., 918180, 918214, 918224]),
 4: array([350007, 350009, 350013, ..., 918229, 918233, 918234]),
 5: array([350201, 350236, 350249, ..., 917881, 917882, 917977]),
 6: array([350004, 350022, 350025, ..., 918196, 918203, 918230]),
 7: array([350035, 350173, 350265, ..., 918075, 918189, 918223])}

In [55]:
create_csv_submission(ids_test, y_test, "output3.csv")

In [73]:
number_of_models = 8

accuracy = []
for train_ind, test_ind in k_fold_split(y=y_train, x=tx_train, n_splits=10):
    x_tr = tx_train[train_ind]
    y_tr = y_train[train_ind]
    x_te = tx_train[test_ind]
    y_te = y_train[test_ind]
    
    new_y_te = np.ones(len(y_te))
    
    ids_tr = ids_train[train_ind]
    ids_te = ids_test[test_ind]
    
    x_train_sep, y_train_sep, ids_train_sep, indx_train_sep = split_data_by_DER_mass_MMC(x_tr, y_tr, ids_tr)
    x_test_sep, y_test_sep, ids_test_sep, indx_test_sep = split_data_by_DER_mass_MMC(x_te, y_te, ids_te)
    
    w = {}
    correct = 0
    for i in range(8):
        x_tr_std, x_te_std = standardize(x_train_sep[i], x_test_sep[i])
        w[i] = least_squares(tx=x_tr_std, y=y_train_sep[i])
        predictions = predict_labels(w[i], x_te_std)
        new_y_te[indx_test_sep[i]] = predictions
        #correct+= np.sum(predictions==y_test_sep[i])
    #accuracy.append(correct*1.0/len(y_te))
    accuracy.append(np.sum(new_y_te == y_te)*1.0/len(y_te))
print (np.std(accuracy))

0.0020906228736910044


In [74]:
np.mean(accuracy)

0.6965640000000001